# Object and scene detection using Amazon Rekognition

***
This notebook provides a walkthrough of [object detection API](https://docs.aws.amazon.com/rekognition/latest/dg/labels.html) in Amazon Rekognition to identify objects.
***

In [1]:
import boto3
import botocore
from IPython.display import HTML, display, Image as IImage
import time

# Initialize clients with default region fallback
REGION = boto3.session.Session().region_name or 'us-east-1'
rekognition = boto3.client('rekognition', region_name=REGION)
s3 = boto3.client('s3', region_name=REGION)

bucket_name = "test-images-891377001208"

# Detect objects in image
***

In [2]:
# Show image
image_name = "cellphone.jpg"
display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucket_name, 'Key': image_name})))

#### Call Rekognition to detect objects in the image

Call Amazon Rekognition to detect objects in the image
<https://docs.aws.amazon.com/rekognition/latest/dg/labels-detect-labels-image.html>

In [3]:
detect_labels_response = rekognition.detect_labels(
    Image={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': image_name,
        }
    }
)

#### Review the raw JSON reponse from Rekognition

In the JSON response below, you will see Label, detected instances, confidence score and additional information.

In [4]:
display(detect_labels_response)

{'Labels': [{'Name': 'Head',
   'Confidence': 99.942138671875,
   'Instances': [],
   'Parents': [{'Name': 'Person'}],
   'Aliases': [],
   'Categories': [{'Name': 'Person Description'}]},
  {'Name': 'Person',
   'Confidence': 99.942138671875,
   'Instances': [],
   'Parents': [],
   'Aliases': [{'Name': 'Human'}],
   'Categories': [{'Name': 'Person Description'}]},
  {'Name': 'Face',
   'Confidence': 99.93607330322266,
   'Instances': [],
   'Parents': [{'Name': 'Head'}, {'Name': 'Person'}],
   'Aliases': [],
   'Categories': [{'Name': 'Person Description'}]},
  {'Name': 'Body Part',
   'Confidence': 99.80298614501953,
   'Instances': [],
   'Parents': [],
   'Aliases': [],
   'Categories': [{'Name': 'Person Description'}]},
  {'Name': 'Finger',
   'Confidence': 99.80298614501953,
   'Instances': [],
   'Parents': [{'Name': 'Body Part'}, {'Name': 'Hand'}, {'Name': 'Person'}],
   'Aliases': [],
   'Categories': [{'Name': 'Person Description'}]},
  {'Name': 'Hand',
   'Confidence': 99.8

#### Display list of detected unsafe objects

You can build your own list of flagged objects.

In [5]:
flagged_objects = ["Cell Phone", "Electronics" "Mobile Phone", "Phone", "Reading"]

for label in detect_labels_response["Labels"]:
    if(label["Name"] in flagged_objects):
        print("Detected unsafe object:")
        print("- {} (Confidence: {})".format(label["Name"], label["Confidence"]))
        print("  - Parents: {}".format(label["Parents"]))

# Recognize objects in video
 Object recognition in video is an async operation. 
https://docs.aws.amazon.com/rekognition/latest/dg/labels-detecting-labels-video.html. 

- First we start a label detection job which returns a Job Id.
- We can then call `get_label_detection` to get the job status and after job is complete, we can get object metadata.
- In production use cases, you would usually use StepFunction or SNS topic to get notified when job is complete.
***

#### Show video in the player

In [10]:
video_name = "objects.mp4"
s3_video_url = s3.generate_presigned_url('get_object', Params={'Bucket': bucket_name, 'Key': video_name})

video_tag = "<video controls='controls' autoplay width='640' height='360' name='Video' src='{0}'></video>".format(s3_video_url)
video_ui = "<table><tr><td style='vertical-align: top'>{}</td></tr></table>".format(video_tag)
display(HTML(video_ui))


""


#### Call Rekognition to start a job for object detection

In [11]:
start_label_detection = rekognition.start_label_detection(
    Video={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': video_name,
        }
    },
)

labels_job_id = start_label_detection['JobId']
display("Job Id: {0}".format(labels_job_id))

'Job Id: 5d0cf12afa005feb8ea4031613f6040fd476de36006c84bb6c504698045d3b46'

### Additional (Optional) Request Attributes

ClientRequestToken, JobTag, MinConfidence, and NotificationChannel:
https://docs.aws.amazon.com/rekognition/latest/APIReference/API_StartLabelDetection.html

Calling Amazon Rekognition Video Operations:
https://docs.aws.amazon.com/rekognition/latest/dg/api-video.html


#### Wait for object detection job to complete

In production use cases, you would usually use StepFunction or SNS topic to get notified when job is complete.

In [12]:
get_object_detection = rekognition.get_label_detection(
    JobId=labels_job_id,
    SortBy='TIMESTAMP'
)

while(get_object_detection['JobStatus'] == 'IN_PROGRESS'):
    time.sleep(5)
    print('.', end='')
 
    get_object_detection = rekognition.get_label_detection(
        JobId=labels_job_id,
        SortBy='TIMESTAMP')
    
display(get_object_detection['JobStatus'])

...

'SUCCEEDED'

#### Review raw JSON reponse from Rekognition

In the JSON response below, you will see list of detected objects and activities.
For each detected object, you will see information like Timestamp.

In [13]:
display(get_object_detection)

{'JobStatus': 'SUCCEEDED',
 'VideoMetadata': {'Codec': 'h264',
  'DurationMillis': 3364,
  'Format': 'QuickTime / MOV',
  'FrameRate': 30.015453338623047,
  'FrameHeight': 850,
  'FrameWidth': 478,
  'ColorRange': 'LIMITED'},
 'Labels': [{'Timestamp': 0,
   'Label': {'Name': 'Adult',
    'Confidence': 98.84953308105469,
    'Instances': [{'BoundingBox': {'Width': 0.9999448657035828,
       'Height': 0.8374617695808411,
       'Left': 5.5175780289573595e-05,
       'Top': 0.16046425700187683},
      'Confidence': 98.80450439453125}],
    'Parents': [{'Name': 'Person'}],
    'Aliases': [],
    'Categories': [{'Name': 'Person Description'}]}},
  {'Timestamp': 0,
   'Label': {'Name': 'Beard',
    'Confidence': 67.09002685546875,
    'Instances': [],
    'Parents': [{'Name': 'Face'}, {'Name': 'Head'}, {'Name': 'Person'}],
    'Aliases': [],
    'Categories': [{'Name': 'Person Description'}]}},
  {'Timestamp': 0,
   'Label': {'Name': 'Black Hair',
    'Confidence': 84.71315002441406,
    'In

#### Display names of recognized objects in the video

In [14]:
the_objects = {}

# Display timestamps and objects detected at that time
str_detail = "Objects detected in video<br>=======================================<br>"
str_overall = "Objects in the overall video:<br>=======================================<br>"

# Objects detected in each frame
for obj in get_object_detection['Labels']:
    ts = obj ["Timestamp"]
    cconfidence = obj['Label']["Confidence"]
    oname = obj['Label']["Name"]
    
    if(oname in flagged_objects):
        print("Found flagged object at {} ms: {} (Confidence: {})".format(ts, oname, round(cconfidence,2)))
    
    str_detail += "At {} ms: {} (Confidence: {})<br>".format(ts, oname, round(cconfidence,2))
    if oname in the_objects:
        cojb = the_objects[oname]
        the_objects[oname] = {"Name" : oname, "Count": 1+cojb["Count"]}
    else:
        the_objects[oname] = {"Name" : oname, "Count": 1}

# Unique objects detected in video
for the_object in the_objects:
    str_overall += "Name: {}, Count: {}<br>".format(the_object, the_objects[the_object]["Count"])

# Display results
display(HTML(str_overall))

In [27]:
list_ui = "<table><tr><td style='vertical-align: top'>{}</td></tr></table>".format(str_detail)
display(HTML(list_ui))

Objects detected in video=======================================


***
### References
- https://docs.aws.amazon.com/rekognition/latest/APIReference/API_DetectLabels.html
- https://docs.aws.amazon.com/rekognition/latest/APIReference/API_StartLabelDetection.html
- https://docs.aws.amazon.com/rekognition/latest/APIReference/API_GetLabelDetection.html

***

You have successfully used Amazon Rekognition to identify specific objects in images and videos.